In [3]:
import torch
import io
import itertools
import os
import os.path as osp
from collections import defaultdict, namedtuple

import dgl
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.sparse as sparse
import sklearn.preprocessing as preprocessing
import torch
import torch.nn.functional as F
from dgl.data.tu import TUDataset
from scipy.sparse import linalg


import argparse
import copy
import os
import time
import warnings

import dgl
import numpy as np
import psutil
import torch
import torch.nn as nn
from joblib import Parallel, delayed
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold
from torch.utils.tensorboard import SummaryWriter

from gcc.contrastive.criterions import NCESoftmaxLoss, NCESoftmaxLossNS
from gcc.contrastive.memory_moco import MemoryMoCo
from gcc.datasets import (
    GRAPH_CLASSIFICATION_DSETS,
    GraphClassificationDataset,
    GraphClassificationDatasetLabeled,
    LoadBalanceGraphDataset,
    NodeClassificationDataset,
    NodeClassificationDatasetLabeled,
    worker_init_fn,
)
from gcc.datasets.data_util import batcher, labeled_batcher
from gcc.models import GraphEncoder
from gcc.utils.misc import AverageMeter, adjust_learning_rate, warmup_linear

from gcc.datasets import data_util


from gcc.utils import rwr




In [12]:
def create_graph_classification_dataset(dataset_name):
    name = {
        "imdb-binary": "IMDB-BINARY",
        "imdb-multi": "IMDB-MULTI",
        "rdt-b": "REDDIT-BINARY",
        "rdt-5k": "REDDIT-MULTI-5K",
        "collab": "COLLAB",
    }[dataset_name]
    dataset = TUDataset(name)
    # dataset.num_labels = dataset.num_labels[0]
    dataset.num_labels = dataset.num_classes
    dataset.graph_labels = dataset.graph_labels.squeeze()
    return dataset

collab = TUDataset("IMDB-BINARY")
print(collab)
print(collab.graph_labels.squeeze().size())
print(collab[0])
print(collab.__dir__())

Dataset("IMDB-BINARY", num_graphs=1000, save_path=/home/shakir/.dgl/IMDB-BINARY_3673667b)
torch.Size([1000])
(Graph(num_nodes=20, num_edges=292,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}), tensor([0]))
['_name', '_url', '_force_reload', '_verbose', '_hash_key', '_hash', '_transform', '_raw_dir', '_save_dir', 'graph_lists', 'graph_labels', 'max_num_node', 'num_labels', '__module__', '__doc__', '__init__', 'process', 'save', 'load', 'has_cache', '__getitem__', '__len__', '_file_path', '_idx_from_zero', '_idx_reset', 'statistics', 'num_classes', 'download', '_download', '_load', '_get_hash', '_get_hash_url_suffix', 'url', 'name', 'raw_dir', 'raw_path', 'save_dir', 'save_path', 'verbose', 'hash', '__repr__', '__dict__', '__weakref__', '__hash__', '__str__', '__getattribute__', '__setattr__', '__delattr__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__new__', '__reduce_ex__', '__reduce_

In [29]:
x = torch.tensor([[1,2,3,4],[5,7,8,9]])
print(x.device)
x = x.cuda(0)
print(x.device)


cpu
cuda:0


In [24]:
dataset = TUDataset( "IMDB-BINARY")
print(dataset.graph_lists[0].__dict__)
# print(dataset.graph_labels.squeeze())

# graph_k = dataset.graph_lists[0]

# graph_k = graph_k.to(torch.device('cuda',1))
# print(graph_k.device)


{'_graph': <dgl.heterograph_index.HeteroGraphIndex object at 0x7f64d4f8ae00>, '_canonical_etypes': [('_N', '_E', '_N')], '_batch_num_nodes': None, '_batch_num_edges': None, '_ntypes': ['_N'], '_is_unibipartite': False, '_srctypes_invmap': {'_N': 0}, '_dsttypes_invmap': {'_N': 0}, '_etypes': ['_E'], '_etype2canonical': {'_E': ('_N', '_E', '_N')}, '_etypes_invmap': {('_N', '_E', '_N'): 0}, '_node_frames': [{'_ID': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])}], '_edge_frames': [{'_ID': tensor([  0,   2,   4,   6,   8,  27,  77,  87, 141, 151,  10,  12,  14,  16,
         18,  20,  22,  24,  29,  97, 125, 173, 201, 237, 253, 269,   1,  11,
         26,  28,  30,  32,  34,  36,  38,  40,  42,  44,  46,  48,  50,  52,
         54,  56,  58,  60,  62,  65,  79,  89,  99, 119, 127, 143, 153, 161,
        175, 195, 203, 217, 225, 239, 255, 271,  31,  64,  66,  68,  70,  72,
         74, 101, 163, 177, 227, 273,   3,  33,  76,  78,  80,  82,  

In [12]:
dataset = data_util.create_graph_classification_dataset('imdb-binary')
graphs = dataset.graph_lists
print(graphs[0])
g4 = graphs[4].to('cuda:1')
print(g4.device)


Graph(num_nodes=20, num_edges=292,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
cuda:1
